## P6 - Q-Learning

The code implements a Q-learning algorithm to find the shortest path in a grid-based environment with rewards and obstacles. The environment consists of an 11x11 grid, where certain cells have negative rewards (aisles and obstacles), one cell has a positive reward (goal), and the rest have default negative rewards. The Q-learning algorithm aims to learn a policy that maximizes the cumulative reward by updating the Q-values for each state-action pair iteratively. The Q-values are initialized to zero, and the algorithm explores the environment by taking actions (up, right, down, left) based on an epsilon-greedy policy. The training process involves updating Q-values using the temporal difference error and a learning rate. After training, the code outputs the learned Q-values and demonstrates the shortest path from a specified starting point (3,9) to the goal by following the actions with the highest Q-values at each step. The epsilon parameter controls the exploration-exploitation trade-off, and the discount factor determines the importance of future rewards.

In [1]:
import numpy as np

actions = ['up', 'right', 'down', 'left']
environment_rows = 11
environment_columns = 11

rewards = np.full((environment_rows, environment_columns), -100.)
rewards[0, 5] = 100.

In [2]:
# aisles depend on your environmnet rows and columns 
aisles = {
    1: list(range(1, 10)),
    2: [1, 7, 9],
    3: list(range(1, 8)) + [9],
    4: [3, 7],
    5: list(range(11)),
    6: [5],
    7: list(range(1, 10)),
    8: [3, 7],
    9: list(range(11))
}

In [3]:
for row_index in range(1, environment_rows - 1):
        for column_index in aisles[row_index]:
            rewards[row_index, column_index] = -1.

q_values = np.zeros((environment_rows, environment_columns, len(actions)))

In [4]:
def is_terminal_state(current_row_index, current_column_index):
    return rewards[current_row_index, current_column_index] != -1.

In [5]:
def get_starting_location():
    while True:
        current_row_index, current_column_index = np.random.randint(
            environment_rows), np.random.randint(environment_columns)
        if not is_terminal_state(current_row_index, current_column_index):
            return current_row_index, current_column_index

In [6]:
def get_next_action(current_row_index, current_column_index, epsilon):
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index, current_column_index])
    else:
        return np.random.randint(len(actions))

In [7]:
def get_next_location(current_row_index, current_column_index, action_index):
    movement = {'up': (-1, 0), 'right': (0, 1),
                'down': (1, 0), 'left': (0, -1)}
    new_row_index = max(0, min(environment_rows - 1,
                                current_row_index + movement[actions[action_index]][0]))
    new_column_index = max(0, min(environment_columns - 1,
                                   current_column_index + movement[actions[action_index]][1]))
    return new_row_index, new_column_index

In [8]:
def get_shortest_path(start_row_index, start_column_index):
    if is_terminal_state(start_row_index, start_column_index):
        return []
    else:
        current_row_index, current_column_index = start_row_index, start_column_index
        shortest_path = [[current_row_index, current_column_index]]

        while not is_terminal_state(current_row_index, current_column_index):
            action_index = get_next_action(
                current_row_index, current_column_index, 1.)
            current_row_index, current_column_index = get_next_location(
                current_row_index, current_column_index, action_index)
            shortest_path.append([current_row_index, current_column_index])

        return shortest_path

In [9]:
epsilon = 0.9
discount_factor = 0.9
learning_rate = 0.9

for episode in range(1000):
    row_index, column_index = get_starting_location()

    while not is_terminal_state(row_index, column_index):
        action_index = get_next_action(row_index, column_index, epsilon)
        old_row_index, old_column_index = row_index, column_index
        row_index, column_index = get_next_location(
            row_index, column_index, action_index)

        reward = rewards[row_index, column_index]
        old_q_value = q_values[old_row_index, old_column_index, action_index]
        temporal_difference = reward + \
            (discount_factor *
            np.max(q_values[row_index, column_index])) - old_q_value
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Environment:')
print(rewards)
print('\nTraining complete!\n')
print('Shortest path: ')
get_shortest_path(3,9)

Environment:
[[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
 [-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
 [-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
 [-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
 [-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]]

Training complete!

Shortest path: 


[[3, 9], [2, 9], [1, 9], [1, 8], [1, 7], [1, 6], [1, 5], [0, 5]]